In [1]:
from ml_battery import *
import sklearn.pipeline
import pickle
import pandas as pd
import numpy as np
import sklearn.ensemble
import sklearn.svm
import sklearn.linear_model
import sklearn.cross_validation
import matplotlib.pyplot as plt

Using TensorFlow backend.
/rufous/occupancy_detection/modules/modules/build/all_of_it/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
from sklearn.datasets import load_iris
data = load_iris()
df = pd.DataFrame(np.hstack((data.data,data.target.reshape((-1,1)))), columns=(data.feature_names+["target"]))
data_train, data_test = sklearn.cross_validation.train_test_split(df, test_size=0.2, random_state=0)

In [3]:
X_train, y_train = data_train.drop("target", axis=1), data_train[["target"]]
X_test, y_test = data_test.drop("target", axis=1), data_test[["target"]]
weights_train, weights_test = np.ones(X_train.shape[0]), np.ones(X_test.shape[0])

In [4]:
codebook = pd.DataFrame(columns=["Name","Type"]).set_index("Name")

In [5]:
clf = sklearn.pipeline.Pipeline([
    ('data_transformation', CodebookTransformer(codebook, X_possible_values = df)),
    ('feature_selection', SelectFromModelPandas(sklearn.ensemble.RandomForestClassifier(n_jobs=-1))),
    ('classification', StackedClassifier(
        MultiClassifier([
            ("random forest", HyperparameterOptimizedEstimator(sklearn.ensemble.RandomForestClassifier(n_jobs=-1), max_depth=(2,10))),
            ("logistic regression", sklearn.linear_model.LogisticRegression(C=100000)),
            ("one-hidden mlp, 1000 epochs, optimized hidden", HyperparameterOptimizedEstimator(OneLayerNNClassifier(), n_hidden=(1,100))),
            ("linear SVC", sklearn.svm.LinearSVC(C=100000)),
            ("naive bayes", MixedNB()),
            ("stratified dummy", PatchedDummy()),
            ("nested logit", NestedClassifier(sklearn.linear_model.LogisticRegression(C=100000), (0, (1,2)))),
        ], parallel=True),
        sklearn.linear_model.LogisticRegression(C=100000))),
])

In [ ]:
clf.fit(X_train, y_train, classification__sample_weight=weights_train)
with open("../models/iris_pipeline_minimal.p","wb") as f:
    pickle.dump(clf,f)

/rufous/occupancy_detection/modules/modules/build/all_of_it/lib/python3.6/site-packages/sklearn/feature_selection/from_model.py:169: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)


In [ ]:
(stacked_scores, stacked_cm, stacked_fi), (multi_scores, multi_cm, multi_fi) = clf.score(X_train, y_train)
multi_scores